# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [68]:
# imports
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [69]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [70]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()

In [71]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [72]:
system_prompt = """
    you are a assisstent specially designed answer the user question. first explain the user technically and then provide the simple \
    non-techinical jargon answer. then provide the user with the example and then explain the example in detail. try to answer the problem\
    in the markdown format with clear distinction of which topic are you explaning.
"""

In [73]:
def get_answer_from_gpt(question: str, system_prompt: str):
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            { "role": "system", "content" : system_prompt},
            { "role" : "user", "content" : question}
        ],
    )
    return response.choices[0].message.content

In [80]:
# Get gpt-4o-mini to answer, with streaming
answer = get_answer_from_gpt(question, system_prompt)
display(Markdown(answer))

### Technical Explanation

The given code snippet utilizes a generator expression in Python that employs the `yield from` statement alongside a set comprehension. Here’s a breakdown of its components:

1. **`{book.get("author") for book in books if book.get("author")}`:**
   - This is a set comprehension that iterates over each `book` in the `books` collection.
   - For each `book`, it retrieves the value associated with the key `"author"` using `book.get("author")`.
   - The comprehension includes a conditional `if book.get("author")` which filters out any books that do not have an author (i.e., it only includes those books where `"author"` is not `None` or an empty string).
   - The output of this comprehension is a set of unique author values.

2. **`yield from`:**
   - This keyword is used in Python to yield values from a generator. It allows the generator function to yield all values produced by another iterable (in this case, the set created by the set comprehension).
   - Using `yield from` helps to streamline the code and allows for cleaner handling of the yielded values, especially if there are multiple items to yield.

### Non-Technical Jargon Explanation

This code is a way to create a list of unique authors from a collection of books. It checks each book to see if there is an author listed, and if there is, it takes that author's name. Then, it sends out all the unique author names found in the list of books.

### Example

Let's say we have a list of books represented as dictionaries:

```python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": "Author A"},  # Duplicate author
    {"title": "Book Four", "author": None},         # No author
    {"title": "Book Five", "author": "Author C"}
]
```

Using the code snippet:

```python
authors = yield from {book.get("author") for book in books if book.get("author")}
```

The output generated by this code would be a set containing: `{'Author A', 'Author B', 'Author C'}`

### Detailed Example Explanation

1. The `books` list contains dictionaries for each book, some of which have authors, and others do not. Notably, "Book Three" has the same author as "Book One" but will only be counted once in the final result.

2. The set comprehension `{book.get("author") for book in books if book.get("author")}` processes the `books` list:
   - For "Book One," it retrieves "Author A."
   - For "Book Two," it retrieves "Author B."
   - For "Book Three," it again retrieves "Author A," but since sets do not allow duplicates, it will not be added again.
   - For "Book Four," it finds that there is no author (returns `None`), so it is skipped.
   - For "Book Five," it retrieves "Author C."

3. The final set of unique authors is `{'Author A', 'Author B', 'Author C'}`.

4. The `yield from` statement sends out each of these unique author names one at a time when accessed, effectively making this a generator that can be iterated over. 

This method of collecting authors ensures not only that all authors are captured but also that there are no duplicates, keeping the process efficient and clean.

In [95]:
# code for the response from the ollma model
def get_answer_from_ollama(question: str, system_prompt: str):
    URL = "http://127.0.0.1:11434/api/chat"
    MODEL = "qwen2.5:latest"
    HEADERS = {"Content-Type": "application/json"}
    
    messages = [
        { "role": "system", "content" : system_prompt},
        { "role" : "user", "content" : question}      
    ]
    
    payload = {
        "model" : MODEL,
        "messages" : messages,
        "stream" : False
    }

    response = requests.post(url, json=payload, headers=HEADERS)
    return (response.json())['message']['content']

In [96]:
# Get Llama 3.2 to answer
answer = get_answer_from_ollama(question, system_prompt)
display(Markdown(answer))

```markdown
## Technical Explanation

### Code Analysis: `yield from {book.get("author") for book in books if book.get("author")}`

This line of Python code is using a generator expression to filter through a list or iterable (`books`), extracting the value of the key `"author"` from each dictionary that contains this key, and then yielding these values.

1. **Generator Expression**:
   - `{book.get("author") for book in books if book.get("author")}`: This is a set comprehension.
   - It iterates over `books`, checks if each item (`book`) has an `"author"` key using the `get` method (which returns `None` if the key does not exist).
   - For each dictionary that contains the `"author"` key, it extracts and adds the value to a set.

2. **`yield from`**:
   - The `yield from` statement is used here to delegate iteration to the generator produced by the comprehension.
   - This means that any caller of this function will get the values as they are generated without needing to wait for all elements to be processed first.

### Simple Non-Technical Explanation

This code looks through a list of books and picks out the names of their authors, but only if each book has an author listed. It then hands these names over one by one to whoever is waiting to receive them.

## Example

Suppose we have a list of dictionaries representing books:

```python
books = [
    {"title": "Book1", "author": "Author1"},
    {"title": "Book2", "author": "Author2"},
    {"title": "Book3"},  # No author here
    {"title": "Book4", "author": "Author3"}
]
```

### Detailed Explanation

1. **Iteration and Filtering**:
   - The code starts by iterating through each dictionary in the `books` list.
   
2. **Checking for Author**:
   - For each book, it checks if the key `"author"` exists using `book.get("author")`.
   - If the key does not exist (`book.get("author")` returns `None`), this book is skipped.

3. **Extracting and Storing Authors**:
   - For books that have an author, their author names are extracted.
   - These names are added to a set (to ensure no duplicate authors).

4. **Yielding the Results**:
   - The `yield from` statement ensures that these author names are passed one by one as they are generated.

### Output

Given the example `books` list, the output would be:

```python
{'Author1', 'Author2', 'Author3'}
```

Each book's author is extracted and added to a set (to avoid duplicates), resulting in a collection of unique authors from the books where this information is available.
```

In summary, the provided code efficiently filters out the authors from a list of books, ensuring no duplicate entries by using a set comprehension and then yields these names one at a time.